# Example tracking workflow

## Ideas
- Cover both Munkres (dense cost matrix) and JV LAPMOD (sparse cost matrix) for linking, comparing speed and memory allocation for different number of points.  This may be one for the talk, rather than having others do it.  That way it doesn't matter what memory is available.


## Installing packages

In [29]:
!pip install --user lap;
!pip install --user matplotlib;
!pip install --user pandas;
!pip install --user Pillow;
!pip install --user scikit-image;

## Importing libraries

In [23]:
import lap
import math
import sys
import util

import numpy as np
from lap import lapmod   

## Getting coordinates present in frame

In [24]:
def get_coordinates(coords, frame):
    # Identifying rows of "coords" with current frame number
    frame_coords = coords[coords['T'] == frame]

    return frame_coords

## Initialise first timepoint
All cells detected in the first timepoint can be considered "new" tracks.  As such, we give them each a unique track ID number.

In [25]:
def initialise_first_timepoint(coords):
    # Adding a blank extra column for the track ID
    coords['TRACK_ID'] = 0

    # Getting the row indices for the first frame
    idx_first = coords['T'] == 0

    coords.loc[idx_first,'TRACK_ID'] = range(1,sum(idx_first)+1)

## Calculate linking costs

In [26]:
def calculate_dense_cost_matrix(prev,curr,thresh):
    # Creating the empty array
    costs = np.empty((len(prev),len(curr)))
    
    # Iterating over each pair, calculating the cost
    for prev_idx,(prev_row,prev_pt) in enumerate(prev.iterrows()):
        for curr_idx,(curr_row,curr_pt) in enumerate(curr.iterrows()):
            # Calculating cost
            cost = calculate_cost(prev_pt,curr_pt,thresh)
            
            # Adding the cost to the cost matrix
            costs[prev_idx,curr_idx] = cost
                
    return costs
    

## Arguments
# idxs1 = "coords" table indices of points in previous frame
# idxs2 = "coords" table indices of points in current frame
# thresh = maximum linking distance for points to move between two frames

## Return
# n_rows = the number of rows in the cost matrix
# cc = array starting at [0,0] and proceeding row-by-row, these are the cost values
# ii = array holding cumulative number of non-zero costs after each row.  Has an extra 0 at the start
# kk = cost matrix column index of each element in cc

def calculate_sparse_cost_matrix(prev,curr,thresh):
    # Initalising variables
    n_NZ = 0
    cc = []
    ii = [0] # ii has an extra 0 at the start
    kk = []
            
    # Iterating over each pair, calculating the cost
    for prev_idx,(prev_row,prev_pt) in enumerate(prev.iterrows()):
        for curr_idx,(curr_row,curr_pt) in enumerate(curr.iterrows()):
            # Calculating cost
            cost = calculate_cost(prev_pt,curr_pt,thresh)
            
            # Only include costs lower than the threshold
            if cost < thresh:
                cc.append(cost)
                kk.append(curr_idx)
                n_NZ = n_NZ + 1
                
        ii.append(n_NZ)
    
    # Converting to Numpy arrays
    cc_np = np.array(cc)
    ii_np = np.array(ii)
    kk_np = np.array(kk)
    
    # Removing columns without values.  Iterate over each possible value, checking if there's an instance of it
    for idx in range(0,kk_np.max()+1):
        # If this value isn't present, shift all remaining columns down by 1
        if np.sum(kk_np==idx) == 0:
            kk_np[kk_np>idx] = kk_np[kk_np>idx]-1
        
    # Remmoving empty rows.  Reverse iterate over all values in ii.  If a value is the same as the previous, remove this row
    for idx in range(len(prev),0,-1):
        if(ii[idx] == ii[idx-1]):
            ii.remove(ii[idx])
            
    ## NEED TO CHECK THERE WILL BE MORE ROWS THAN COLUMNS, OTHERWISE TRANSPOSE COST MATRIX AND RETURN BOOLEAN INDICATING IF 
    ## IT'S BEEN TRANSPOSED

    return len(ii)-1, cc_np, np.array(ii), kk_np
            
    
def calculate_cost(prev_pt,curr_pt,thresh):
    # Spatial linking (distance between two points)
    dx = (curr_pt.X-prev_pt.X)
    dy = (curr_pt.Y-prev_pt.Y)
    d = math.sqrt(dx*dx + dy*dy)
    
    # If the two points are separated by more than the linking threshold, set them to infinity ('inf')
    if d > thresh:
        d = math.inf
    
    return d

## Main workflow

In [28]:
# Setting parameters
np.set_printoptions(precision=3,threshold=sys.maxsize)
linking_thresh = 50

# # Loading image stack
# path = "..\\data\\ExampleTimeseries.tif"
# images = util.load_images(path);

# Loading coordinates
path = "../data/TestObjectCoordinatesNoHeader.csv"
coords = util.load_coordinates(path);

# # Getting the number of frames
# n_frames = images.shape[2]
# print("Loaded %i frames" % n_frames)

# Set new track IDs for each object in the first frame
initialise_first_timepoint(coords)

# Starting at frame 2, looping over each frame, linking pairs
for frame in range(1,2): #range (1,n_frames):
    # Get coordinates from the previous frame
    prev_coords = get_coordinates(coords,frame-1)
    
    # Get coordinates from the current frame
    curr_coords = get_coordinates(coords,frame)
    
    print("Prev:\n",prev_coords,"\n")    
    print("Curr:\n",curr_coords,"\n")
    
    # Calculate costs for each possible link
    dense_costs = calculate_dense_cost_matrix(prev_coords,curr_coords,linking_thresh)
    print("Dense costs:\n",dense_costs,"\n")
    
    n_rows, cc, ii, kk = calculate_sparse_cost_matrix(prev_coords,curr_coords,linking_thresh)
    cc[2] = 0.1
    print("Sparse costs:\n",(n_rows, cc, ii, kk),"\n")
    
    # Calculate assigninments for each
    opt,x,y = lapmod(n_rows,cc,ii,kk)
    
    print(opt)
    print("x = %a" % x)
    print("y = %a" % y)

Loading coordinates from " ../data/TestObjectCoordinatesNoHeader.csv "
Loaded data shape:  (16, 6)
 
Prev:
     ID           X           Y    T         AREA  INTENSITY  TRACK_ID
0  1.0   24.000000  499.000000  0.0  34061.72464       69.0         1
1  2.0   44.000000  581.000000  0.0  33106.98446      193.0         2
2  3.0   51.209726  516.325228  0.0  33761.68693      329.0         3
3  4.0   76.464126  464.414798  0.0  33698.22197      446.0         4
4  5.0  114.000000  523.000000  0.0  33975.46230      305.0         5
5  6.0  121.203349  467.595694  0.0  33847.26077      418.0         6
6  7.0  166.000000  568.000000  0.0  33848.50820      305.0         7 

Curr:
       ID           X           Y    T         AREA  INTENSITY  TRACK_ID
7   44.0   25.000000  495.000000  1.0  34052.37681       69.0         0
8   45.0   44.000000  581.000000  1.0  33118.20725      193.0         0
9   46.0   55.960422  513.007916  1.0  33789.78100      379.0         0
10  47.0   76.592992  461.223720  1